In [1]:
import os
import cv2
import math
import types
import random
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from torch.autograd import Variable
from torchvision import datasets, transforms, models
from sklearn.metrics import classification_report
from torch import optim
import time
from torch.utils.data import Dataset, DataLoader

In [2]:
# parameters setting
target_shape = [3,96,96]
num_classes = 7
num_layers = 5
cut_size = 225
init_lr = 0.0003
batch_size = 128
use_illumi_correct = False
device= torch.device("cuda")

In [3]:
# load data
data_dir = ''

train_data = torch.load(data_dir + 'pickles/AffectNet_train_full_p1.pt')
train_img, train_y = train_data['images'], train_data['labels']

test_data = torch.load(data_dir + 'pickles/AffectNet_test.pt')
test_img, test_y = test_data['images'], test_data['emotion']

all_perm = torch.tensor(np.load(data_dir+'permutations_1000.npy'))

In [4]:
def nine_crops(image, size=[96,96], n_crop=3):
    xx = yy = n_crop
    x = size[0] // xx
    y = size[1] // yy
    crops = ()
    for j in range(yy):
        for i in range(xx):
            left = i*x
            up = y*j
            right = left + x
            low = up + y
            region = image.crop((left,up,right,low))
            region = transforms.RandomCrop(64)(region)
            region = transforms.Resize((75, 75), Image.BILINEAR)(region)
#             region = transforms.Lambda(rgb_jittering)(region)
            crops += (region,)       
    return crops


# def rgb_jittering(im):
#     im = np.array(im, 'int32')
#     for ch in range(3):
#         im[:, :, ch] += np.random.randint(-2, 2)
#     im[im > 255] = 255
#     im[im < 0] = 0
#     return im.astype('uint8')


class PuzzleCrop(object):
    def __init__(self, size, n_crop):
        self.size = size
        self.n_crop = n_crop

    def __call__(self, img):
        return nine_crops(img, self.size, self.n_crop)


class Lambda(object):
    def __init__(self, lambd):
        assert isinstance(lambd, types.LambdaType)
        self.lambd = lambd
    def __call__(self, img):
        return self.lambd(img)

In [5]:
from torch.utils.data import Dataset, DataLoader
#======= data loaders =======#    
n_crop = 3
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([int(cut_size*1.1),int(cut_size*1.1)]),
    transforms.RandomCrop([cut_size,cut_size]),
    transforms.RandomHorizontalFlip(),
    PuzzleCrop([cut_size,cut_size],3),
    Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops]))
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    PuzzleCrop([cut_size,cut_size],3),
    Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops]))
])

resize_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([cut_size,cut_size]),
    transforms.ToTensor()
])

def default_loader(images):
    img_tensor = train_transform(images)
    return img_tensor

def default_test_loader(images):
    img_tensor = test_transform(images)
    return img_tensor

class Dataset(Dataset):
    def __init__(self,imgs,permutations,loader=default_loader):
        self.images = imgs 
        self.permutations = permutations
        self.loader = loader

    def __getitem__(self, index):

        img = self.images[index]
        n_imgs = tmp = self.loader(img)
        img = resize_transform(img)
        label = torch.randint(0,1000,(1,))
        for i in range(self.permutations.size(1)):
            idx = self.permutations[label][0][i]
            n_imgs[i] = tmp[idx]
        return n_imgs,label

    
    def __len__(self):
        return len(self.images)

In [15]:
class Network(nn.Module):

    def __init__(self, lr=0.001,classes=1000):
        super(Network, self).__init__()

        
        alexnet = models.alexnet(pretrained=True)
        self.conv = alexnet.features
        
        self.fc6 = nn.Sequential()
        self.fc6.add_module('fc6_s1',nn.Linear(256, 512))
        self.fc6.add_module('relu6_s1',nn.ReLU(inplace=True))
        self.fc6.add_module('drop6_s1',nn.Dropout(p=0.5))
        self.classifier = nn.Sequential(
            nn.Linear(4608,4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096,classes)
        )
        
        self.loss_fn = nn.CrossEntropyLoss()
#         self.optimizer=torch.optim.Adam([p for p in self.parameters() if p.requires_grad] , lr=lr)
        self.optimizer=torch.optim.SGD(self.parameters(),lr=lr,momentum=0.9,weight_decay = 5e-4)
    
    def forward(self, x):
        B,T,C,H,W = x.size()
        x = x.transpose(0,1)
        x_list = []
        for i in range(9):
            z = self.conv(x[i])
            z = self.fc6(z.view(B,-1))
            z = z.view([B,1,-1])
            x_list.append(z)
        x = torch.cat(x_list,1)
        x = self.classifier(x.view(B,-1))
        return x
    
    def train_model(self, data_loader, num_epochs, lr=0.0001, min_loss=5.):
        self.train()
        start=time.time()

        for epoch in range(num_epochs):
            running_loss=0
            running_acc = 0.
            total = 0
            num_batches = 0
            for count, (images, label) in enumerate(data_loader):
                self.optimizer.zero_grad()
                
                images=images.to(device)
                label=torch.squeeze(label).to(device)

                scores = self.forward(images)

                loss =  self.loss_fn(scores, label) 
                loss.backward()

                self.optimizer.step()

                running_loss += loss.detach().item()

                acc = (scores.argmax(-1) == label).float().sum()
                running_acc += acc.item()

                total+=images.size(0)
                num_batches += 1
            total_loss = running_loss/num_batches
            if total_loss<min_loss:
                min_loss = total_loss
                torch.save(model.state_dict(),data_dir+'baselines/Jigsaw_AlexNet_AffectNet_minloss.pth')
                print('save model', min_loss)
            total_acc = running_acc/total
            elapsed = time.time()-start
            print('\t', 'epoch=',epoch, '\t time = {:.0f}m {:.0f}s'.format(elapsed // 60, elapsed % 60),'\t lr=', lr,'\t loss = ', total_loss , '\t Acc = {:.6f}'.format(total_acc*100),'%')
        return total_loss
    
    def test_model(self, data_loader):
        self.eval()
        start=time.time()
        
        running_loss = 0.
        running_acc = 0
        total = 0
        num_batches = 0
        
        preds = []
        truth = []
        
        for count, (images, label) in enumerate(data_loader):
            self.optimizer.zero_grad()
            
            images = images.to(device)

            scores = self.forward(x).detach().cpu()

            acc = (scores.argmax(-1) == label).float().sum()
            running_acc += acc.item()
            
            total += images.size(0)

            preds.extend(scores.argmax(-1).numpy())
            truth.extend(label.numpy())
            
        total_acc = running_acc/total
        elapsed = time.time()-start

        print('testing','\t time = {:.0f}m {:.0f}s'.format(elapsed // 60, elapsed % 60), '\n Acc = {:.6f}'.format(total_acc*100),'%')
        test_fscore = classification_report(truth, preds)
        print(test_fscore)

In [16]:
train_input = Dataset(train_img,all_perm)
train_loader = DataLoader(train_input, batch_size=batch_size,shuffle=True)

test_input = Dataset(test_img,all_perm)
test_loader = DataLoader(test_input, batch_size=batch_size,shuffle=False)

model = Network(lr=0.001,classes=1000)
model.to(device)
print(model)
LR=0.001
loss = 10.
for i in range(1):
    print('Iteration', i)
    loss = model.train_model(data_loader=train_loader, num_epochs=30,lr=LR, min_loss=loss)
    LR = LR*0.1

Network(
  (conv): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc6): Sequential(
    (fc6_s1): Linear(in_features=256, out_features=512, bias=True)
    (relu6_s1): ReLU(inplace)
    (drop6_s1): Dropout(p=0.5)
  )
  (classifier): Sequential(
    (0): Linear(in_features=460